# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data = pd.read_csv("cities.csv")
city_data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,luanda,3,AO,1563902863,83,-8.83,13.24,73.40,5.82
1,1,dikson,5,RU,1563902930,90,73.51,80.55,34.72,2.95
2,2,hithadhoo,99,MV,1563902930,76,-0.60,73.08,83.32,7.61
3,3,codrington,0,AU,1563902819,87,-28.95,153.24,55.99,9.17
4,4,tiznit,7,MA,1563902930,92,29.70,-9.73,69.10,11.77


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:

gmaps.configure(api_key=g_key)

In [4]:
humidity = city_data["Humidity"]
loc = city_data[["Lat", "Lng"]]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig 


NameError: name 'gmaps' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
small_city = city_data.loc[(city_data["Max Temp"] < 80) & (city_data["Max Temp"] > 70) \
                                    & (city_data["Wind Speed"] < 10) \
                                    & (city_data["Cloudiness"] == 0)].dropna()
small_city = small_city.dropna()
small_city

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
43,43,goderich,0,CA,1563902935,54,43.74,-81.71,75.99,8.99
97,97,athabasca,0,CA,1563902942,50,54.72,-113.29,79.18,5.21
180,180,lusambo,0,CD,1563902953,26,-4.97,23.44,74.68,2.10
215,215,praia,0,BR,1563902957,38,-20.25,-43.81,77.00,5.82
280,280,bandrele,0,YT,1563902966,69,-12.91,45.19,77.00,9.17
303,303,paldiski,0,EE,1563902969,77,59.35,24.08,71.60,9.17
396,396,wana,0,PK,1563902981,65,32.31,69.59,71.62,1.88
427,427,esmeraldas,0,BR,1563902925,38,-19.76,-44.31,77.00,5.82
471,471,zhezkazgan,0,KZ,1563902991,42,47.80,67.71,73.06,8.61
525,525,sitka,0,US,1563902998,51,37.17,-99.65,77.00,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = small_city[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

In [ ]:

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
 # Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig